In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget https://mirrors.estointernet.in/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

--2021-07-22 09:38:46--  https://mirrors.estointernet.in/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving mirrors.estointernet.in (mirrors.estointernet.in)... 43.255.166.254, 2403:8940:3:1::f
Connecting to mirrors.estointernet.in (mirrors.estointernet.in)|43.255.166.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/octet-stream]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz.2’

spark-3.1.2-bin-had 100%[===================>] 218.23M  25.4MB/s    in 9.4s    

2021-07-22 09:38:56 (23.3 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz.2’ saved [228834641/228834641]



In [3]:
!tar -xvzf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

In [4]:
!ls /content/spark-3.1.2-bin-hadoop3.2
!pip install findspark

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]= "/content/spark-3.1.2-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark 3.0 on Google Colab").getOrCreate()

In [8]:
spark

In [9]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('KMeans_clustering_algorithm')

In [10]:
data= spark.read.csv("/content/sample_data/lotto_summary.csv", header=True, inferSchema=True)

In [11]:
data.show()

+--------------------+---+---------+---------+----------+---+------------+--------+--------------------+--------------+---+---+----------+--------------------+--------------------+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|          device_aid|  1| latitude|longitude| timestamp|  5|manufacturer|   model|                   8|            ip| 10| 11|        12|                  13|                  14|  15|  16|  17|  18|   19|  20|  21|  22|  23|  24|  25|  26|  27|  28|  29|  30|  31|  32|  33|  34|  35|  36|  37|
+--------------------+---+---------+---------+----------+---+------------+--------+--------------------+--------------+---+---+----------+--------------------+--------------------+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|b7cb746e-668d-463...|  0|31.370023|75.380034|1582134058| 34|        null|    null|                null|     

Finding the missing values

In [12]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----------+---+--------+---------+---------+---+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|device_aid|  1|latitude|longitude|timestamp|  5|manufacturer|model|  8| ip| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30| 31| 32| 33| 34| 35| 36| 37|
+----------+---+--------+---------+---------+---+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|         1|  1|       1|        1|        1|  1|         217|  217|217|217|  1|  1|  1|239|239|251|251|239|239|239|251|251|251|251|251|251|251|251|251|217|251|251|251|251|251|251|250|251|
+----------+---+--------+---------+---------+---+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+



In [13]:
print("Distinct Count: " + str(data.distinct().count()))

Distinct Count: 252


In [14]:
df=data.dropna('all')
df.show()

+--------------------+---+---------+---------+----------+---+------------+--------+--------------------+--------------+---+---+----------+--------------------+--------------------+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|          device_aid|  1| latitude|longitude| timestamp|  5|manufacturer|   model|                   8|            ip| 10| 11|        12|                  13|                  14|  15|  16|  17|  18|   19|  20|  21|  22|  23|  24|  25|  26|  27|  28|  29|  30|  31|  32|  33|  34|  35|  36|  37|
+--------------------+---+---------+---------+----------+---+------------+--------+--------------------+--------------+---+---+----------+--------------------+--------------------+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|b7cb746e-668d-463...|  0|31.370023|75.380034|1582134058| 34|        null|    null|                null|     

Drop all value which is having 1 missing values


In [15]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----------+---+--------+---------+---------+---+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|device_aid|  1|latitude|longitude|timestamp|  5|manufacturer|model|  8| ip| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30| 31| 32| 33| 34| 35| 36| 37|
+----------+---+--------+---------+---------+---+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|         0|  0|       0|        0|        0|  0|         216|  216|216|216|  0|  0|  0|238|238|250|250|238|238|238|250|250|250|250|250|250|250|250|250|216|250|250|250|250|250|250|249|250|
+----------+---+--------+---------+---------+---+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+



In [16]:
print("Distinct Count: " + str(df.distinct().count()))

Distinct Count: 251


Removing the unwanted columns
and the col which is having most of missing values

In [17]:
columns_to_drop = ['15','16','20','21','22','23','24','25','26','27','28','30','31','32','33','34','35','36','37']
df= df.drop(*columns_to_drop)

In [18]:
df.show()

+--------------------+---+---------+---------+----------+---+------------+--------+--------------------+--------------+---+---+----------+--------------------+--------------------+----+----+-----+----+
|          device_aid|  1| latitude|longitude| timestamp|  5|manufacturer|   model|                   8|            ip| 10| 11|        12|                  13|                  14|  17|  18|   19|  29|
+--------------------+---+---------+---------+----------+---+------------+--------+--------------------+--------------+---+---+----------+--------------------+--------------------+----+----+-----+----+
|b7cb746e-668d-463...|  0|31.370023|75.380034|1582134058| 34|        null|    null|                null|          null| in|280|tttd7qdpdh|                null|                null|null|null| null|null|
|96739989-8b83-407...|  0|30.595877|75.699551|1581983032|  6|        vivo|    1802|          in_jio_jio|157.39.138.226| in|268|ttmy49f9zk|com.search.gurban...|    Gurbani Searcher|   a| 8.1|PH

In [19]:
pip install pyspark

In [20]:
import pyspark

In [21]:
from pyspark import SparkContext
from pyspark import SparkConf

In [22]:
from pyspark.sql.functions import to_timestamp

converted the timestamp


In [23]:
df.withColumn("timestamp",to_timestamp("timestamp")) \
  .show(truncate=False)

+------------------------------------+---+---------+---------+-------------------+---+------------+--------+-----------------------------+--------------+---+---+----------+--------------------------------+-------------------------------------------------+----+----+-----+----+
|device_aid                          |1  |latitude |longitude|timestamp          |5  |manufacturer|model   |8                            |ip            |10 |11 |12        |13                              |14                                               |17  |18  |19   |29  |
+------------------------------------+---+---------+---------+-------------------+---+------------+--------+-----------------------------+--------------+---+---+----------+--------------------------------+-------------------------------------------------+----+----+-----+----+
|b7cb746e-668d-4631-b1d5-e43a5f466756|0  |31.370023|75.380034|2020-02-19 17:40:58|34 |null        |null    |null                         |null          |in |280|tttd7qdp